In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.cm as cm
from matplotlib import pyplot as plt 
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input
from keras import backend as K
from keras.engine.topology import Layer

# packages for learning from crowds
from crowd_layer.crowd_layers import CrowdsClassification, MaskedMultiCrossEntropy
from crowd_layer.crowd_aggregators import CrowdsCategoricalAggregator

# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
NUM_RUNS = 30
DATA_PATH = "/Users/yangyajing/Documents/noisy_dataset/LabelMe/prepared/"
N_CLASSES = 8
BATCH_SIZE = 64
N_EPOCHS = 50

In [3]:
def load_data(filename):
    f = open(filename, 'rb')
    data = np.load(f)
    f.close()
    return data

In [4]:
print("\nLoading train data...")

# images processed by VGG16
data_train_vgg16 = load_data(DATA_PATH+"data_train_vgg16.npy")
print(data_train_vgg16.shape)

# ground truth labels
labels_train = load_data(DATA_PATH+"labels_train.npy")
print(labels_train.shape)

# labels obtained from majority voting
labels_train_mv = load_data(DATA_PATH+"labels_train_mv.npy")
print(labels_train_mv.shape)

# labels obtained by using the approach by Dawid and Skene
labels_train_ds = load_data(DATA_PATH+"labels_train_DS.npy")
print(labels_train_ds.shape)

# data from Amazon Mechanical Turk
print("\nLoading AMT data...")
answers = load_data(DATA_PATH+"answers.npy")
print(answers.shape)
N_ANNOT = answers.shape[1]
print("\nN_CLASSES:", N_CLASSES)
print("N_ANNOT:", N_ANNOT)

# load test data
print("\nLoading test data...")

# images processed by VGG16
data_test_vgg16 = load_data(DATA_PATH+"data_test_vgg16.npy")
print(data_test_vgg16.shape)

# test labels
labels_test = load_data(DATA_PATH+"labels_test.npy")
print(labels_test.shape)


Loading train data...
(10000, 4, 4, 512)
(10000,)
(10000,)
(10000,)

Loading AMT data...
(10000, 59)

N_CLASSES: 8
N_ANNOT: 59

Loading test data...
(1188, 4, 4, 512)
(1188,)


In [5]:
def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets

In [6]:
print("\nConverting to one-hot encoding...")
labels_train_bin = one_hot(labels_train, N_CLASSES)
print(labels_train_bin.shape)
labels_train_mv_bin = one_hot(labels_train_mv, N_CLASSES)
print(labels_train_mv_bin.shape)
labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
print(labels_train_ds_bin.shape)
labels_test_bin = one_hot(labels_test, N_CLASSES)
print(labels_test_bin.shape)

answers_bin_missings = []
for i in range(len(answers)):
    row = []
    for r in range(N_ANNOT):
        if answers[i,r] == -1:
            row.append(-1 * np.ones(N_CLASSES))
        else:
            row.append(one_hot(answers[i,r], N_CLASSES)[0,:])
    answers_bin_missings.append(row)
answers_bin_missings = np.array(answers_bin_missings).swapaxes(1,2)
answers_bin_missings.shape


Converting to one-hot encoding...
(10000, 8)
(10000, 8)
(10000, 8)
(1188, 8)


(10000, 8, 59)

In [7]:
answers_test_bin_missings = np.zeros((len(labels_test), N_CLASSES))
answers_test_bin_missings[np.arange(len(labels_test)), labels_test] = 1
answers_test_bin_missings = np.repeat(answers_test_bin_missings.reshape([len(labels_test),N_CLASSES,1]), N_ANNOT, axis=2)
answers_test_bin_missings.shape

(1188, 8, 59)

In [8]:
def eval(model,y_test):
    return dict(zip(model.metrics_names,model.evaluate(data_test_vgg16,y_test, verbose=False)))

In [11]:
hidden_layers = Sequential()
hidden_layers.add(Flatten(input_shape=data_train_vgg16.shape[1:]))
hidden_layers.add(Dense(128, activation='relu'))
hidden_layers.add(Dropout(0.5))

train_inputs = Input(shape=(data_train_vgg16.shape[1:]))
last_hidden = hidden_layers(train_inputs)
baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)

# add crowds layer on top of the base model
channeled_output = CrowdsClassification(N_CLASSES, N_ANNOT, conn_type="MW")(baseline_output)
print(channeled_output)
simple_model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])

# instantiate specialized masked loss to handle missing answers
loss = MaskedMultiCrossEntropy().loss

# compile model with masked loss and train
simple_model.compile(optimizer='adam', 
                     loss=[loss,'categorical_crossentropy'], 
                     loss_weights=[1,0],
                     metrics=['accuracy']
                    )

Tensor("baseline_1/Softmax:0", shape=(?, 8), dtype=float32)
<tf.Variable 'crowds_classification_2/CrowdLayer:0' shape=(8, 8, 59) dtype=float32_ref>
Tensor("crowds_classification_2/Reshape_2:0", shape=(?, 8, 59), dtype=float32)


In [14]:
simple_model.fit(data_train_vgg16, [answers_bin_missings, labels_train_mv_bin], epochs=N_EPOCHS, shuffle=True, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/50
10000/10000 [==============================] - 3s - loss: 0.0712 - crowds_classification_2_loss: 0.0712 - baseline_loss: 2.0077 - crowds_classification_2_acc: 0.0240 - baseline_acc: 0.6029     
Epoch 2/50
10000/10000 [==============================] - 2s - loss: 0.0621 - crowds_classification_2_loss: 0.0621 - baseline_loss: 1.3414 - crowds_classification_2_acc: 0.0214 - baseline_acc: 0.7543     
Epoch 3/50
10000/10000 [==============================] - 3s - loss: 0.0570 - crowds_classification_2_loss: 0.0570 - baseline_loss: 1.2790 - crowds_classification_2_acc: 0.0189 - baseline_acc: 0.7910     
Epoch 4/50
10000/10000 [==============================] - 2s - loss: 0.0529 - crowds_classification_2_loss: 0.0529 - baseline_loss: 1.2403 - crowds_classification_2_acc: 0.0178 - baseline_acc: 0.8072     
Epoch 5/50
10000/10000 [==============================] - 2s - loss: 0.0490 - crowds_classification_2_loss: 0.0490 - baseline_loss: 1.2485 - crowds_classification_2_acc: 0.0188 - b

10000/10000 [==============================] - 2s - loss: 0.0170 - crowds_classification_2_loss: 0.0170 - baseline_loss: 0.9694 - crowds_classification_2_acc: 0.0184 - baseline_acc: 0.8736     
Epoch 41/50
10000/10000 [==============================] - 2s - loss: 0.0171 - crowds_classification_2_loss: 0.0171 - baseline_loss: 0.9539 - crowds_classification_2_acc: 0.0182 - baseline_acc: 0.8711     
Epoch 42/50
10000/10000 [==============================] - 2s - loss: 0.0167 - crowds_classification_2_loss: 0.0167 - baseline_loss: 0.9359 - crowds_classification_2_acc: 0.0177 - baseline_acc: 0.8708     
Epoch 43/50
10000/10000 [==============================] - 2s - loss: 0.0165 - crowds_classification_2_loss: 0.0165 - baseline_loss: 0.9446 - crowds_classification_2_acc: 0.0181 - baseline_acc: 0.8740     
Epoch 44/50
10000/10000 [==============================] - 2s - loss: 0.0160 - crowds_classification_2_loss: 0.0160 - baseline_loss: 0.9632 - crowds_classification_2_acc: 0.0185 - baseline

In [15]:
eval(simple_model,y_test=[answers_test_bin_missings,labels_test_bin])

{'baseline_acc': 0.8291245791245792,
 'baseline_loss': 1.1054960393945779,
 'crowds_classification_2_acc': 0.034722222222222224,
 'crowds_classification_2_loss': 0.9964053753249171,
 'loss': 0.9964053753249171}

In [10]:
channeled_output

<tf.Tensor 'crowds_classification_1/Reshape_2:0' shape=(?, 8, 59) dtype=float32>